In [40]:
import pandas as pd
import nltk
import urllib
import numpy as np
import re

In [41]:
Master = pd.read_csv('LoughranMcDonald_MasterDictionary_2016.csv', index_col=False, header=0);
Constrain = pd.read_excel('constraining_dictionary.xlsx')
Uncertain = pd.read_excel('uncertainty_dictionary.xlsx')
data = pd.read_csv('StopWords.txt', header = None)
dfs = pd.read_excel('cik_list.xlsx')

In [42]:
stop=[data.iloc[i,0] for i in range(len(data))]
stop.append("nbsp")

In [43]:
positive=(Master[Master["Positive"]>0])
negative=(Master[Master['Negative']>0])

In [44]:
positive_dict=[i.lower() for i in positive["Word"] if i.lower() not in stop]


In [45]:
negative_dict=[i.lower() for i in negative["Word"] if i.lower() not in stop]

In [46]:
constrain_dict=[i.lower() for i in Constrain["Word"]]

In [47]:
uncertain_dict=[i.lower() for i in Uncertain["Word"]]

In [48]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [49]:
pattern=re.compile("[a-zA-z]+[.']*[a-zA-z]?")


In [50]:
ans=[]

In [51]:
def getSyllables(word):
    vowel="aeiou"
    count=0
    for i in word:
        if i in vowel:
            count=count+1
    if(word[-1]=="d" and word[-2]=='e'):
        count=count-1
        
    if(word[-1]=='s' and word[-2]=='e'):
        count=count-1
        
    return count

In [52]:
def getVariables(txt):
    positive_score=0
    negative_score=0
    polarity_score=0
    average_sentence_length=0
    percentage_of_complex_words=0
    fog_index=0
    complex_word_count=0
    word_count=len(txt)
    uncertainty_score=0
    constraining_score=0
    positive_word_proportion=0
    negative_word_proportion=0
    uncertainty_word_proportion=0
    constraining_word_proportion=0
    sentences=0
    
    for i in txt:
        if i in positive_dict:
            positive_score=positive_score+1
            
        if i in negative_dict:
            negative_score=negative_score+1
            
        if i in constrain_dict:
            constraining_score=constraining_score+1
            
        if i in uncertain_dict:
            uncertainty_score=uncertainty_score+1
    
        if i[-1]=='.':
            sentences=sentences+1
            
            
        x=getSyllables(i)
        
        if(x>2):
            complex_word_count=complex_word_count+1
    
    
    polarity_score= (positive_score-negative_score)/((positive_score+negative_score)+0.000001)
    
    if(word_count>0):
        
        average_sentence_length=word_count/sentences
        
        percentage_of_complex_words= complex_word_count/word_count
        
        fog_index=0.4 *( percentage_of_complex_words + average_sentence_length )
        
        
        positive_word_proportion=positive_score/word_count
        
        negative_word_proportion= negative_score/word_count
        
        constraining_word_proportion= constraining_score/word_count
        
        uncertainty_word_proportion= uncertainty_score/word_count
        
        
    z=[positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count,word_count,uncertainty_score,constraining_score,positive_word_proportion,negative_word_proportion,uncertainty_word_proportion,constraining_word_proportion]
        
    return z   
        
        
        
    
    
            

In [53]:
for i in range(len(dfs)):
    txt = urllib.request.urlopen("https://www.sec.gov/Archives/"+dfs.iloc[i,5]).read()
    txt=txt.decode("utf-8") 
    txt=remove_tags(txt)
    txt=pattern.findall(txt)
    clean_txt=[j for j in txt if j.lower() not in stop]
    
    temp=[]
    flag1=0
    flag2=0
    flag3=0
    j=0
    MDA=[]
    QQDMR=[]
    RF=[]
    while(j<len(clean_txt)):
        if(clean_txt[j]=="risk"):
            if(clean_txt[j+1]=="factors" and clean_txt[j-2]=="item"):
                flag1=1
                j=j+2
        if(clean_txt[j]=="item"):
            flag1=0
        if(flag1==1):
            RF.append(clean_txt[j].lower())
            
            
            
        
        if(clean_txt[j]=="MANAGEMENT'S"):
            if(clean_txt[j+1]=="DISCUSSION" and clean_txt[j+2]=="ANALYSIS" and clean_txt[j-1]=="ITEM"):
                flag2=1
                j=j+4
        if(clean_txt[j]=="ITEM"):
            flag2=0
        if(flag2==1):
            MDA.append(clean_txt[j].lower())
            
            
            
            
            
        if(clean_txt[j]=="QUANTITATIVE"):
            if(clean_txt[j+1]=="QUALITATIVE" and clean_txt[j+2]=="DISCLOSURE" and clean_txt[j-1]=="ITEM"):
                flag3=1
                j=j+4
        if(clean_txt[j]=="ITEM"):
            flag3=0
        if(flag3==1):
            QQDMR.append(clean_txt[j].lower())
        j=j+1 
    
    
    MDA_variables=getVariables(MDA)
    QQDMR_variables=getVariables(QQDMR)
    RF_variables=getVariables(RF)
    
    
   # print(MDA_variables,QQDMR_variables,RF_variables)
    temp=MDA_variables+QQDMR_variables+RF_variables
    temp= list(dfs.iloc[i,:])  + temp
    ans.append(temp)
    print(i+1,"/",dfs.shape[0])
    

1 / 152
2 / 152
3 / 152
4 / 152
5 / 152
6 / 152
7 / 152
8 / 152
9 / 152
10 / 152
11 / 152
12 / 152
13 / 152
14 / 152
15 / 152
16 / 152
17 / 152
18 / 152
19 / 152
20 / 152
21 / 152
22 / 152
23 / 152
24 / 152
25 / 152
26 / 152
27 / 152
28 / 152
29 / 152
30 / 152
31 / 152
32 / 152
33 / 152
34 / 152
35 / 152
36 / 152
37 / 152
38 / 152
39 / 152
40 / 152
41 / 152
42 / 152
43 / 152
44 / 152
45 / 152
46 / 152
47 / 152
48 / 152
49 / 152
50 / 152
51 / 152
52 / 152
53 / 152
54 / 152
55 / 152
56 / 152
57 / 152
58 / 152
59 / 152
60 / 152
61 / 152
62 / 152
63 / 152
64 / 152
65 / 152
66 / 152
67 / 152
68 / 152
69 / 152
70 / 152
71 / 152
72 / 152
73 / 152
74 / 152
75 / 152
76 / 152
77 / 152
78 / 152
79 / 152
80 / 152
81 / 152
82 / 152
83 / 152
84 / 152
85 / 152
86 / 152
87 / 152
88 / 152
89 / 152
90 / 152
91 / 152
92 / 152
93 / 152
94 / 152
95 / 152
96 / 152
97 / 152
98 / 152
99 / 152
100 / 152
101 / 152
102 / 152
103 / 152
104 / 152
105 / 152
106 / 152
107 / 152
108 / 152
109 / 152
110 / 152
111 / 15

In [54]:
import csv

with open("output.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(ans)